In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:
import os
import numpy as np
import glob
import csv
import keras
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten


from inception_model_generator import MRNet_inc_model
from vgg_model_generator import  MRNet_vgg_model
import tensorflow as tf

from numpy.random import seed
seed(1)
tf.random.set_seed(2)

Using TensorFlow backend.


In [3]:
b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)
data_gen = MRNet_data_generator(data_path, p, l, batch_size=b_size, scale_to=(299,299), model="inception")

model , cp_callback= MRNet_inc_model(["abnormal", "axial"])


# print(x.shape)
model.fit(x=data_gen, epochs=3, verbose=True, callbacks=[cp_callback], shuffle=False, use_multiprocessing=True, workers=6)
# model.save_weights("training_inc_1/w.ckpt")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 2)                 23092898  
Total params: 23,092,898
Trainable params: 23,055,138
Non-trainable params: 37,760
_________________________________________________________________
Epoch 1/3
   6/1130 [..............................] - ETA: 2:05:45 - loss: 0.0921 - binary_accuracy: 1.0000

Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Process ForkPoolWorker-6:
  File "/usr/lib/python3.6/multiproc

Epoch 1/3


  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: ignored